In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['outcomes.csv', 'patient_data_4830_hours.csv', 'datasets_og.pickle']


In [ ]:
import pandas as pd
import numpy as np
import glob, os, sys
import time
import pickle
import datetime

In [ ]:
df = pd.read_csv('../input/patient_data_4830_hours.csv')
df.head(50)

In [ ]:
matrix3D = np.array(df.drop(['SUBJECT_ID', 'TimeStamp'], 1))
matrix3D

In [ ]:
matrix3D.shape

In [ ]:
matrix3D = np.array(matrix3D).reshape((6587, 98, 37))
matrix3D.shape

In [ ]:
g = []
for k in range(37):
    h = (sum(sum(matrix3D[:,:,k])))
    if int(h)==0:
        g.append(False)
    else:
        g.append(True)
g = np.array(g)
g

In [ ]:
matrix3D = matrix3D[:,:,g]
matrix3D.shape

In [ ]:
outcomes = pd.read_csv('../input/outcomes.csv')
outcomes.head(10)

In [ ]:
Y = np.array(outcomes.drop(['SUBJECT_ID'], 1))
Y.shape

In [ ]:
X = matrix3D
X.shape

In [ ]:
X = np.concatenate((X,X))
X.shape

In [ ]:
Y = np.concatenate((Y,Y))
Y.shape

In [ ]:
X = np.concatenate((X,X))
X = np.concatenate((X,X))
X = np.concatenate((X,X))
display(X.shape)
Y = np.concatenate((Y,Y))
Y = np.concatenate((Y,Y))
Y = np.concatenate((Y,Y))
Y.shape

In [ ]:
X_new = X.reshape(X.shape[0],1,X.shape[1],X.shape[2])
X_new.shape

In [ ]:
Y_new = Y.reshape(Y.shape[0],1,Y.shape[1])
print(Y_new.shape)

In [ ]:
X_new = np.insert(X_new, 1, Y_new, axis=1)
X_new.shape

In [ ]:
X_new.shape

In [ ]:
np.random.shuffle(X_new)
X_new.shape

In [ ]:
X = X_new[:,0]

X.shape

In [ ]:
Y = X_new[:,1]
Y = Y[:,0]
Y = Y[:,0]
Y = Y.reshape(Y.shape[0],1)
Y.shape

In [ ]:
X_train = X[:95000]
X_test = X[95000:]
Y_train = Y[:95000]
Y_test = Y[95000:]
display(X_train.shape)
display(X_test.shape)
display(Y_train.shape)
display(Y_test.shape)

In [ ]:
sum(Y_train)

In [ ]:
sum(Y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Bidirectional,BatchNormalization

In [ ]:
modelb = Sequential()
modelb.add(LSTM(100, input_shape=(X_train.shape[1],X_train.shape[2]),return_sequences=True))
modelb.add(Bidirectional(LSTM(25),merge_mode='sum'))
modelb.add(Dropout(0.2))
#modelb.add(BatchNormalization(axis=1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
modelb.add(Dense(100,activation='tanh'))
modelb.add(Dense(25,activation='tanh'))
modelb.add(Dense(10,activation='tanh'))
modelb.add(Dense(1, activation='sigmoid'))
modelb.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(modelb.summary())
historyb = modelb.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30,batch_size=1000,verbose=1)

In [ ]:
scoresb = modelb.evaluate(X_test, Y_test)
scoresb[1]*100

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
import pandas as pd;
import numpy as np;
from sklearn import model_selection
from sklearn.metrics import roc_auc_score, f1_score,precision_recall_fscore_support
from sklearn.metrics import confusion_matrix

In [ ]:
y_pred = modelb.predict(X_test)

In [ ]:
score = 1.0 - roc_auc_score(Y_test, y_pred)



display(score)


f_score=f1_score(Y_test,y_pred.round())

display(f_score)





prfs=precision_recall_fscore_support(Y_test,y_pred.round())


display(prfs)




f_score_micro=f1_score(Y_test,y_pred.round(),average='micro')
f_score_macro=f1_score(Y_test,y_pred.round(),average='macro')

display(f_score_micro)
display(f_score_macro)




cf=confusion_matrix(Y_test,y_pred.round())

display(cf)




print("AUC : "+str(1.0 - score)+",  Score/Loss : "+str(score)+", F1_Score_average: "+str(f_score)+", Precision, Recall, F1_score, Support: "+str(prfs))
print("F1_score_micro: "+str(f_score_micro))
print("F1_score_macro: "+str(f_score_macro))
print("Confusion Matrix")
print(cf)
print("END\n\n")



In [ ]:



# In[271]:


fpr = dict()
tpr = dict()
roc_auc = dict()
fpr[1], tpr[1], _ = roc_curve(Y_test[:, 0], y_pred[:, 0])
roc_auc[1] = auc(fpr[1], tpr[1])


# In[272]:


display(roc_auc)


# In[275]:


plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(historyb.history['acc'])
plt.plot(historyb.history['val_acc'])
plt.title('Model accuracy BLSTM')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(historyb.history['loss'])
plt.plot(historyb.history['val_loss'])
plt.title('Model loss BLSTM')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
ans = []
anss = []
histories = []
class LSTM(object):
    def __init__(self):
        # define lower bound of benchmark function
        self.Lower = 0
        # define upper bound of benchmark function
        self.Upper = 1

    # function which returns evaluate function
    def function(self):
        def evalute(D,sol):
            from keras.models import Sequential
            from keras.layers import Dense
            from keras.layers import LSTM
            from keras.layers import Dropout
            sol = np.array(sol)
            sol = sigmoid(sol)
            #display(sol)
            op = sol>=0.5
            X_tr = X_train[:,:,op]
            X_te = X_test[:,:,op]
            modelf = Sequential()
            modelf.add(LSTM(100, input_shape=(X_tr.shape[1],X_tr.shape[2]),return_sequences=True))
            modelf.add(LSTM(50))
            modelf.add(Dropout(0.2))
            #modelf.add(BatchNormalization())
            modelf.add(Dense(100,activation='tanh'))
            modelf.add(Dense(25,activation='tanh'))
            modelf.add(Dense(10,activation='tanh'))
            modelf.add(Dense(1, activation='sigmoid'))
            modelf.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
            history = modelf.fit(X_tr, Y_train, validation_data=(X_te, Y_test), epochs=30,batch_size=1000,verbose=1)
            scores = modelf.evaluate(X_te, Y_test, verbose=0)
            anss.append(scores[1])
            histories.append(history)
            k = -1*scores[1]
            return k
        return evalute

In [ ]:
X_tr = X_train.reshape(X_train.shape[0]*X_train.shape[1],X_train.shape[2])
display(X_tr.shape)
X_train.shape

In [ ]:
X_te = X_test.reshape(X_test.shape[0]*X_test.shape[1],X_test.shape[2])
display(X_te.shape)
X_test.shape

In [ ]:
dfTrain = pd.DataFrame()

In [ ]:
len(X_tr[:,0])

In [ ]:
for i in range(0,X_tr.shape[1]):
    dfTrain['col_'+str(i+1)] = pd.Series(X_tr[:,i])
dfTrain.head()

In [ ]:
dfTrain.shape

In [ ]:
dfTest = pd.DataFrame()

In [ ]:
for i in range(0,X_te.shape[1]):
    dfTest['col_'+str(i+1)] = pd.Series(X_te[:,i])
dfTest.head()

In [ ]:
dfTest.shape

In [ ]:
dfYtrain = pd.DataFrame()
dfYtrain['label'] = pd.Series(Y_train[:,0])
dfYtrain.head()

In [ ]:
dfYtrain.shape

In [ ]:
dfYtest = pd.DataFrame()
dfYtest['label'] = pd.Series(Y_test[:,0])
dfYtest.head()

In [ ]:
dfYtest.shape

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "data.csv"):  
    print('1')
    csv = df.to_csv(index=False)
    print('2')
    b64 = base64.b64encode(csv.encode())
    print('3')
    payload = b64.decode()
    print('4')
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    print('5')
    html = html.format(payload=payload,title=title,filename=filename)
    print('6')
    return HTML(html)

In [ ]:
create_download_link(df = dfTrain,filename='Xtrain.csv')

In [ ]:
indices = list(chunk_marks(dfm.shape[0], 100))
list

In [ ]:
w = create_download_link(df = dfTest,filename='Xtest.csv')
w

In [ ]:
create_download_link(df = dfYtrain,filename='Ytrain.csv')

In [ ]:
create_download_link(df = dfYtest,filename='Ytest.csv')